In [7]:
!wget -O onnxruntime_gpu-1.11.0-cp36-cp36m-linux_aarch64.whl https://nvidia.box.com/shared/static/pmsqsiaw4pg9qrbeckcbymho6c01jj4z.whl

--2024-08-22 15:39:17--  https://nvidia.box.com/shared/static/pmsqsiaw4pg9qrbeckcbymho6c01jj4z.whl
Resolving nvidia.box.com (nvidia.box.com)... 74.112.186.157
Connecting to nvidia.box.com (nvidia.box.com)|74.112.186.157|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/pmsqsiaw4pg9qrbeckcbymho6c01jj4z.whl [following]
--2024-08-22 15:39:17--  https://nvidia.box.com/public/static/pmsqsiaw4pg9qrbeckcbymho6c01jj4z.whl
Reusing existing connection to nvidia.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://nvidia.app.box.com/public/static/pmsqsiaw4pg9qrbeckcbymho6c01jj4z.whl [following]
--2024-08-22 15:39:17--  https://nvidia.app.box.com/public/static/pmsqsiaw4pg9qrbeckcbymho6c01jj4z.whl
Resolving nvidia.app.box.com (nvidia.app.box.com)... 74.112.186.157
Connecting to nvidia.app.box.com (nvidia.app.box.com)|74.112.186.157|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loca

In [8]:
!pip3 install ./onnxruntime_gpu-1.11.0-cp36-cp36m-linux_aarch64.whl

Processing ./onnxruntime_gpu-1.11.0-cp36-cp36m-linux_aarch64.whl


In [1]:
import onnxruntime as ort
import numpy as np
import torch

In [2]:
ort_model = ort.InferenceSession("drl_forward_opset16.onnx", providers=['CUDAExecutionProvider', 'TensorrtExecutionProvider'])

Set up the camera

In [3]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.VBox([widgets.HBox([image])]))

Action scaling like in training

In [4]:
def rescale_actions(low, high, action):
    d = (high - low) / 2.0
    m = (high + low) / 2.0
    scaled_action =  action * d + m
    return scaled_action

In [5]:
from jetbot import Robot

In [6]:
robot = Robot()

Main loop

In [7]:
def update(change):
    image = change['new']
    image = image.astype(np.float32)
    image = np.expand_dims(image, axis=0) 
    # processed_image = preprocess(image)
    outputs = ort_model.run(None, {"obs": image})
    mu = torch.tensor(outputs[0][0])
    #vprint("mu")
    #vprint(mu)
    post_processed_mu = rescale_actions(-.1, .1, torch.clamp(mu, -1.0, 1.0))
    # print("processed mu")
    print(post_processed_mu)
    robot.set_motors(post_processed_mu[0].item(), post_processed_mu[1].item())
    

In [8]:
camera.observe(update, names='value')

tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])
tensor([0.1000, 0.1000])


In [ ]:
camera.unobserve(update, names='value')
camera_link.unlink()
camera.stop()
robot.stop()


In [1]:
robot.stop()

NameError: name 'robot' is not defined

In [5]:
ort_model

NameError: name 'ort_model' is not defined

In [ ]:

device = torch.device('cuda')
import torchvision.transforms as transforms
import PIL.Image
import numpy as np

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    return image[None, ...]
